In [1]:
import os
import sys
import argparse
import glob
import numpy as np
import pickle
import json
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
from projection_utils import Projection
from data_parser import read_keypoints

sys.path.insert(1, '../')
from virtual_huams_resource.utils import get_smplx_body_model

sys.path.insert(0, "../virtual_huams_resource")
print(sys.path)
from virtual_huams_resource.benji_prox_dataloader import DatasetBase, proxDatasetSkeleton

INFO - 2022-05-28 13:59:03,007 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


['../virtual_huams_resource', '/Users/guangzhu/Desktop/ETHz/S2/Virtual_Human/project/prox/src', '../', '/Users/guangzhu/.vscode/extensions/ms-toolsai.jupyter-2022.4.1021342353/pythonFiles', '/Users/guangzhu/.vscode/extensions/ms-toolsai.jupyter-2022.4.1021342353/pythonFiles/lib/python', '/Users/guangzhu/miniconda3/envs/VH/lib/python39.zip', '/Users/guangzhu/miniconda3/envs/VH/lib/python3.9', '/Users/guangzhu/miniconda3/envs/VH/lib/python3.9/lib-dynload', '', '/Users/guangzhu/miniconda3/envs/VH/lib/python3.9/site-packages']


In [2]:
data_root = '../quantitative'
sequence_id = 'vicon_03301_01'
save_root = '../quantitative/3d_visualization'

smplx_model_path='../models_smplx_v1_1/models/smplx/SMPLX_NEUTRAL2.pkl'

In [3]:
# ______________ read frames ______________
rgb_list = glob.glob(os.path.join(data_root, "recordings" , sequence_id, 'Color/*.jpg'))
rgb_list.sort()
# rgb_list[-1][-33:-30]
n_frame = int(rgb_list[-1][-33:-30])  # total frame number
# rgb_list
n_frame
# rgb_list[-1][-34:-4]

10

In [4]:
# ______________ init ______________
projection = Projection(os.path.join(data_root, "calibration"))
MAX_DEPTH = 20.0
depth_scale = 1e3       # TODO: determine depth_scale
point_cloud = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()
h_rgb = 1080
w_rgb = 1920
h_depth = 424
w_depth = 512

curr_frame = rgb_list[6]

In [5]:
# ______________ read depth image ______________
depth_img = cv2.imread(os.path.join(data_root, "recordings", sequence_id, 'Depth/' + curr_frame[-34:-4] + '.png'), flags=-1).astype(float)
# depth_raw = depth_img.copy()
depth_img /= 8.0
depth_img /= 1000.0
# depth_img = cv2.flip(depth_img, 1)
# depth_raw = cv2.flip(depth_raw, 1)
# depth_aligned = projection.align_depth2color(depth_img, depth_raw)

# ______________ create point cloud ______________
# depth_cam = projection.depth_cam;
# depth_intrinsic = o3d.camera.PinholeCameraIntrinsic(w_depth, h_depth, depth_cam['f'][0], depth_cam['f'][1], depth_cam['c'][0], depth_cam['c'][1])
# depth_extrinsic = depth_cam['view_mtx']
# o3d.geometry.PointCloud.create_from_depth_image(depth_img, depth_intrinsic, depth_extrinsic, );

In [6]:
# visualize 3D scene mesh
scene_mesh = o3d.io.read_triangle_mesh(os.path.join(data_root, "scenes", "vicon.ply"))
# o3d.visualization.draw_geometries([scene_mesh])

In [7]:
# visualize 3D scene point cloud
scene_point_cloud = o3d.io.read_point_cloud(os.path.join(data_root, "scenes", "N3OpenArea.ply"))
scene_point_cloud
# o3d.visualization.draw_geometries([scene_point_cloud])

PointCloud with 624597 points.

In [8]:
# visualize skeleton in 3D point cloud
body_model = get_smplx_body_model(smplx_model_path)
path = os.path.join(data_root, "fittings/mosh", sequence_id, "results/s001_frame_00001__00.00.00.023/000.pkl")
with open(path, 'rb') as file:
    human_mesh = pickle.load(file, encoding='latin1')
human_mesh

{'transl': array([[-0.6994415 , -0.31427762,  0.9848179 ]], dtype=float32),
 'num_pca_comps': 12,
 'beta': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'jaw_pose': array([[0., 0., 0.]], dtype=float32),
 'body_pose': array([[ 0.0590809 ,  0.05310533,  0.09144919,  0.06638564, -0.03938352,
         -0.03405466,  0.04605372,  0.03528766, -0.00767644, -0.01093746,
         -0.04850643, -0.04170513, -0.00294363,  0.09496448,  0.04342466,
          0.04438157, -0.05772606,  0.02723523,  0.02083636,  0.204744  ,
         -0.04182123,  0.00783336, -0.13903025,  0.09834542,  0.03948856,
         -0.01690864,  0.0097099 ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.11294897,  0.02547551,
          0.01097451,  0.05551571, -0.02713786, -0.38216677,  0.06716956,
          0.01517425,  0.40331653,  0.11045438, -0.00737314,  0.05088175,
          0.0304325 , -0.12942097, -0.5711655 ,  0.06874575,  0.0668682 ,
          0.5659935 ,  0.

In [9]:
pd2 = proxDatasetSkeleton(root_dir=data_root + '/temp', in_frames=15, pred_frames=30, output_type='raw_pkls',
                        search_prefix='results', extra_prefix='joints_worldnorm.pkl', frame_jump=10, window_overlap_factor=8)
# skeleton = proxDatasetSkeleton(pd2)
# skeleton.__getitem__(0)

In [10]:
idx, (a, in_skels), (b, fut_skels) = pd2.__getitem__(0)
a

['../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00001__00.00.00.031/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00011__00.00.00.321/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00021__00.00.00.654/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00031__00.00.00.985/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00041__00.00.01.323/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00051__00.00.01.654/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00061__00.00.01.986/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00071__00.00.02.318/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001_frame_00081__00.00.02.653/joints_worldnorm.pkl',
 '../quantitative/temp/N3OpenArea_00157_02/results/s001

In [11]:
in_skels[0].shape

torch.Size([1, 25, 3])

In [12]:
# in_skels[0]
skeleton = in_skels[0].detach().numpy()
skeleton = skeleton.astype(np.float64)
skeleton = skeleton.squeeze()

In [13]:
# visualize skeleton in 3D
LIMBS = [(23, 15),
         (24, 15),
         (15, 22),
         (22, 12),
         # left arm
         (12, 13),
         (13, 16),
         (16, 18),
         (18, 20),
         # right arm
         (12, 14),
         (14, 17),
         (17, 19),
         (19, 21),
         # spline
         (12, 9),
         (9, 6),
         (6, 3),
         (3, 0),
         # left leg
         (0, 1),
         (1, 4),
         (4, 7),
         (7, 10),
         # right leg
         (0, 2),
         (2, 5),
         (5, 8),
         (8, 11)]

skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(skeleton),
            o3d.utility.Vector2iVector(LIMBS))

# o3d.visualization.draw_geometries([skeleton_input])
# vis.add_geometry(skeleton_input)

In [14]:
# ctr = vis.get_view_control()
# cam_param = ctr.convert_to_pinhole_camera_parameters()
# cam_param = update_cam(cam_param, trans)
# ctr.convert_from_pinhole_camera_parameters(cam_param)

# vis.poll_events()
# vis.update_renderer()

vis.create_window()
vis.add_geometry(scene_point_cloud)
vis.add_geometry(skeleton_input)
render_option = vis.get_render_option()
render_option.line_width = 1000
vis.update_renderer()
vis.run()
vis.destroy_window()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 